In [2]:

from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.docstore.document import Document
from typing import List, Dict
from langchain.vectorstores import Redis
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
    AttributeInfo)
from dotenv import load_dotenv
load_dotenv()
import os

In [4]:
from __future__ import annotations

from pathlib import Path
from typing import Any, Callable, Dict, Iterable, List, Optional

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import Field
from langchain_core.retrievers import BaseRetriever
import bm25s

def default_preprocessing_func(text: str) -> List[str]:
    token_corpus = bm25s.tokenize(texts=text, stopwords = "vi", return_ids= False , show_progress=False)
    return token_corpus


class BM25SRetriever(BaseRetriever):
    """A toy retriever that contains the top k documents that contain the user query.

    This retriever only implements the sync method _get_relevant_documents.

    If the retriever were to involve file access or network access, it could benefit
    from a native async implementation of `_aget_relevant_documents`.

    As usual, with Runnables, there's a default async implementation that's provided
    that delegates to the sync implementation running on another thread.
    """
    vectorizer: Any
    """ BM25S vectorizer."""
    docs: List[Document] = Field(repr=False)
    """List of documents to retrieve from."""
    k: int = 4
    """Number of top results to return"""
    preprocess_func: Callable[[str], List[str]] = default_preprocessing_func
    """ Preprocessing function to use on the text before BM25 vectorization."""
    save_directory : Optional[str] = None
    """ Directory for saving BM25S index."""
    activate_numba: bool = False
    """Accelerate backend"""
    class Config:
        arbitrary_types_allowed = True
    @classmethod
    def from_texts(
        cls,
        texts: Iterable[str],
        metadatas: Optional[Iterable[dict]] = None,
        bm25_params: Optional[Dict[str, Any]] = None,
        save_directory : Optional[str] = save_directory,
        preprocess_func: Callable[[str], List[str]] = default_preprocessing_func,
        **kwargs: Any,
    ) -> BM25SRetriever:
        """
        Create a BM25Retriever from a list of texts.
        Args:
            texts: A list of texts to vectorize.
            metadatas: A list of metadata dicts to associate with each text.
            bm25s_params: Parameters to pass to the BM25s vectorizer.
            preprocess_func: A function to preprocess each text before vectorization.
            **kwargs: Any other arguments to pass to the retriever.

        Returns:
            A BM25SRetriever instance.
        """
        try:
            from bm25s import BM25
        except ImportError:
            raise ImportError(
                "Could not import bm25s, please install with `pip install "
                "bm25s`."
            )
        bm25_params = bm25_params or {}
        if save_directory and Path(save_directory).exists():
            try:
                vectorizer = BM25.load(save_directory)
            except Exception as e:
                print(f"Failed to load BM25 index from {save_directory}: {e}")
                print("Proceeding with indexing from scratch.")
                texts_processed = preprocess_func(texts)
                vectorizer = BM25(**bm25_params)
                vectorizer.index(texts_processed)
                if save_directory:
                    vectorizer.save(save_directory)

        else:
            texts_processed = preprocess_func(texts)
            vectorizer = BM25(**bm25_params)
            vectorizer.index(texts_processed)
            if save_directory:
                vectorizer.save(save_directory)

        metadatas = metadatas or ({} for _ in texts)
        docs = [Document(page_content=t, metadata=m) for t, m in zip(texts, metadatas)]
        return cls(
            vectorizer=vectorizer, docs=docs, preprocess_func=preprocess_func, save_directory=save_directory, **kwargs
        )

    @classmethod
    def from_documents(
        cls,
        documents: Iterable[Document],
        *,
        bm25_params: Optional[Dict[str, Any]] = None,
        preprocess_func: Callable[[str], List[str]] = default_preprocessing_func,

        **kwargs: Any,
    ) -> BM25SRetriever:
        """
        Create a BM25Retriever from a list of Documents.
        Args:
            documents: A list of Documents to vectorize.
            bm25_params: Parameters to pass to the BM25 vectorizer.
            preprocess_func: A function to preprocess each text before vectorization.
            **kwargs: Any other arguments to pass to the retriever.

        Returns:
            A BM25Retriever instance.
        """
        texts, metadatas = zip(*((d.page_content, d.metadata) for d in documents))
        return cls.from_texts(
            texts=texts,
            bm25_params=bm25_params,
            metadatas=metadatas,
            preprocess_func=preprocess_func,
            **kwargs,
        )

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        processed_query = self.preprocess_func(query)
        if self.activate_numba : 
          self.vectorizer.activate_numba_scorer()
          return_docs = self.vectorizer.retrieve(processed_query, k=self.k, backend_selection="numba")
          return [self.docs[i] for i in return_docs.documents[0]]
        else :  
          return_docs, scores = self.vectorizer.retrieve(processed_query, self.docs, k = self.k)
          return [return_docs[0, i] for i in range(return_docs.shape[1])]


In [3]:
from langchain_community.vectorstores import Qdrant
from langchain_core.documents import Document
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain.retrievers import EnsembleRetriever
from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.docstore.document import Document
from typing import List, Dict
from langchain.chains import RetrievalQA
import os

HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1,api_key= os.getenv('llm_api_1'))


# Function to load and chunk data from a folder
def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder
    for filename in os.listdir(data_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(data_path, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            docs.extend(loader.load())

    # Define headers to split on for Markdown splitting
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    # Initialize Markdown splitter
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )

    chunk_size = 512
    chunk_overlap = 0

    # Initialize character-based splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    chunked_docs = []

    # Process each document
    for doc in docs:
        # Split document by Markdown headers
        md_header_splits = markdown_splitter.split_text(doc.page_content)

        # Further split the Markdown header splits into smaller chunks
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs


data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/thongtintuyensinh'
chunked_data = load_and_chunk_data(data_path)


vectorstore = Qdrant.from_documents(
    chunked_data,
    HF_EMBEDDING,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="tintuyensinh",
)

/home/justtuananh/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/justtuananh/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`

In [39]:
import datetime
from typing import Literal, Optional, Tuple, Union
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

class MultiDataSourceSearch(BaseModel):
    """Search over multi databases about a university."""

    db_description: Optional[str] = Field(
        None,
        description="Description for the datasource selection",
    )
    db_name: Union[
        Literal["Student's book"], 
        Literal["School's introduction"], 
        Literal["Admission information"]
    ] = Field(
        ...,
        description="The name of the datasource.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            value = getattr(self, field)
            if value is not None and value != self.__fields__[field].default:
                print(f"{field}: {value}")
from langchain_core.prompts import ChatPromptTemplate

system = """You are an expert at converting user questions into database queries. \
Given a question, return a database query optimized to retrieve the most relevant results.
Student's book : 
If there are acronyms or words you are not familiar with, do not try to rephrase them.
The return value must be in Vietnamese

If none of the database is not sutable, return `regular`
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatGroq(model_name="llama3-groq-8b-8192-tool-use-preview", temperature=0,api_key= "")
structured_llm = llm.with_structured_output(MultiDataSourceSearch)
query_analyzer = prompt | structured_llm

In [53]:
query_analyzer.invoke({"question": "hi hi"}).pretty_print()

db_name: Student's book


In [44]:
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field


from langchain_core.prompts import ChatPromptTemplate

system = """Task type
Task: Check if there is unsafe content in 'User' messages in conversations according to our safety policy with the below categories.

Policy
<BEGIN UNSAFE CONTENT CATEGORIES>
S1: Violent Crimes.
S2: Non-Violent Crimes
S3: Sex-Related Crimes
S4: Child Sexual Exploitation
S5: Defamation
S6: Specialized Advice
S7: Privacy
S8: Intellectual Property
S9: Indiscriminate Weapon
S10: Hate
S11: Suicide & Self-Harm
S12: Sexual Content
S13: Elections
S14: Offensive and Vulgar Language
<END UNSAFE CONTENT CATEGORIES>


Output format
Provide your safety assessment for ONLY THE LAST User message in the above conversation:

First line must read 'safe' or 'unsafe'.
If unsafe, a second line must include a comma-separated list of violated categories.
"""
prompt_safe = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
from dotenv import load_dotenv
load_dotenv()
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1,api_key= os.getenv('llm_api_1'))

question_save = prompt_safe | llm 

out = question_save.invoke({"question": "huớng dẫn cách lách luật của học viện"})
out.content

In [52]:
out = question_save.invoke({"question": "huớng dẫn cách lách luật của học viện"})
out.content

'unsafe\nS2'

In [26]:
question = 'năm 2024 lấy bao nhiêu nữ'
result = router.invoke({"question": question})
result

RouteQuery(datasource='sotaysinhvien')

In [19]:
retriever_bm25 = BM25SRetriever.from_documents(chunked_data, k= 20, activate_numba = True)
retriever_qdrant = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 20})

BM25S Create Vocab:   0%|          | 0/101 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/101 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/101 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/101 [00:00<?, ?it/s]

In [20]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever_bm25, retriever_qdrant], weights=[0.2, 0.8]
)

In [65]:

docs = ensemble_retriever.invoke("Năm 2024 học viện lấy bao nhiêu chỉ tiêu nữ")
docs

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(metadata={'Header 1': 'THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024', 'Header 2': '4\\.Chỉ tiêu tuyển sinh'}, page_content='## 4\\.Chỉ tiêu tuyển sinh  \nThực hiện theo Quyết định của Bộ Quốc phòng giao. Chỉ tiêu tuyển sinh được xác định theo giới tính (thí sinh nam, thí sinh nữ) và vùng, miền (miền Bắc được tính từ Quảng Bình trở ra, miền Nam được tính từ Quảng Trị trở vào). Chỉ tiêu tuyển sinh năm 2024 sẽ được Học viện thông báo chi tiết sau khi Bộ Quốc phòng giao chỉ tiêu chính thức. Các đối tượng tuyển sinh gồm:\nĐối tượng tuyển sinh: Đào tạo kỹ sư quân sự (Chỉ huy, quản lý kỹ thuật)  \nMã trường: KQH\nMã ngành: 7860220'),
 Document(metadata={'Header 1': 'THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024', 'Header 2': '1\\. Đối tượng tuyển sinh'}, page_content='\\- Nam, nữ thanh niên ngoài quân đội (kể cả quân nhân đã xuất ngũ và công dân hoàn thành nghĩa vụ công an nhân dân).'),
 Document(metadata={'Header 1': 'THÔNG TIN KỲ TUYỂN SINH CỦA H

In [66]:
print(len(docs))

31


[Document(metadata={'Header 1': 'THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024', 'Header 2': '4\\.Chỉ tiêu tuyển sinh'}, page_content='## 4\\.Chỉ tiêu tuyển sinh  \nThực hiện theo Quyết định của Bộ Quốc phòng giao. Chỉ tiêu tuyển sinh được xác định theo giới tính (thí sinh nam, thí sinh nữ) và vùng, miền (miền Bắc được tính từ Quảng Bình trở ra, miền Nam được tính từ Quảng Trị trở vào). Chỉ tiêu tuyển sinh năm 2024 sẽ được Học viện thông báo chi tiết sau khi Bộ Quốc phòng giao chỉ tiêu chính thức. Các đối tượng tuyển sinh gồm:\nĐối tượng tuyển sinh: Đào tạo kỹ sư quân sự (Chỉ huy, quản lý kỹ thuật)  \nMã trường: KQH\nMã ngành: 7860220'),
 Document(metadata={'Header 1': 'THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024', 'Header 2': '1\\. Đối tượng tuyển sinh'}, page_content='\\- Nam, nữ thanh niên ngoài quân đội (kể cả quân nhân đã xuất ngũ và công dân hoàn thành nghĩa vụ công an nhân dân).'),
 Document(metadata={'Header 1': 'THÔNG TIN KỲ TUYỂN SINH CỦA H

In [71]:
docs_new = []
for doc in docs: 
    docs_new.append(doc.page_content)

In [15]:
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
from langchain.retrievers.document_compressors import CohereRerank
import os
import cohere
from dotenv import load_dotenv
load_dotenv()
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))
query = "Năm 2024 học viện lấy bao nhiêu chỉ tiêu nữ"


information_list = [
    "Ngày sinh của Albert Einstein: 14 tháng 3, 1879",
    "Thủ đô của Nhật Bản: Tokyo",
    "Tốc độ ánh sáng: 299,792,458 mét/giây",
    "Đơn vị tiền tệ của Mỹ: Đô la Mỹ (USD)",
    "Người phát minh ra bóng đèn: Thomas Edison",
    "Ngôn ngữ chính thức của Brazil: Tiếng Bồ Đào Nha",
    "Chất liệu làm từ sợi thiên nhiên phổ biến nhất: Bông",
    "Thời gian để Trái Đất quay quanh Mặt Trời: 365 ngày",
    "Tác giả của tiểu thuyết 'Harry Potter': J.K. Rowling",
    "Kim tự tháp Giza được xây dựng ở quốc gia nào: Ai Cập"
]

# Câu hỏi liên quan đến một trong các thông tin
question = "Ai là người phát minh ra bóng đèn?"

resultss = co.rerank(model="rerank-multilingual-v3.0", query=question, documents=information_list, top_n=2, return_documents=True)

results = dict(resultss)['results']

# Lấy giá trị của từng document từ list 'results'
documents = [item.document.text for item in results]

# In ra các document
for doc in documents:
    print(doc)


print("-----------------------------------")

# from langchain_community.document_transformers import LongContextReorder

# Reorder the documents:
# Less relevant document will be at the middle of the list and more
# relevant elements at beginning / end.
# reordering = LongContextReorder()
# reordered_docs = reordering.transform_documents(results)
# reordered_docs


Người phát minh ra bóng đèn: Thomas Edison
Ngày sinh của Albert Einstein: 14 tháng 3, 1879
-----------------------------------


In [ ]:
context = " ".join(results[0].document["text"].split(" ")[-4096:])


# check retrieaval

In [32]:
from langchain_community.vectorstores import Qdrant
from langchain_qdrant import QdrantVectorStore
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from tqdm import tqdm
import os

HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")

def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder and its subfolders
    for root, _, files in os.walk(data_path):
        for filename in files:
            if filename.endswith('.txt'):
                file_path = os.path.join(root, filename)
                loader = TextLoader(file_path, encoding='utf-8')
                docs.extend(loader.load())

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=True
    )

    chunk_size = 512
    chunk_overlap = 0

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    chunked_docs = []

    for doc in docs:
        md_header_splits = markdown_splitter.split_text(doc.page_content)
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs




/tmp/ipykernel_14162/1995128018.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/data/thongtintuyensinh'
docs = load_and_chunk_data(data_path)

In [30]:
from langchain_qdrant import QdrantVectorStore
from langchain_community.vectorstores import Qdrant
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from dotenv import load_dotenv
import os
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain_qdrant import RetrievalMode
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

load_dotenv()
embeddings = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

arabic.txt:   0%|          | 0.00/6.35k [00:00<?, ?B/s]

catalan.txt:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

chinese.txt:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

danish.txt:   0%|          | 0.00/424 [00:00<?, ?B/s]

...

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

slovene.txt:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

romanian.txt:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

tajik.txt:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [31]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="thongtintuyensinh_db",
    sparse_embedding=sparse_embeddings,
    url = os.getenv('URL_QDRANT'),
    api_key=os.getenv('API_QDRANT'),
    retrieval_mode=RetrievalMode.SPARSE,
)

QdrantVectorStoreError: Existing Qdrant collection thongtintuyensinh_db does not contain sparse vectors named {}. If you want to recreate the collection, set `force_recreate` parameter to `True`.

In [ ]:
# Điểm trúng tuyển cho thí sinh nữ có thường trú phía Nam năm 2021 là bao nhiêu? 
# Năm nào thí sinh nữ có thường trú phía Bắc có điểm trúng tuyển cao nhất vào Học viện Kỹ thuật Quân sự?
# Năm nào học viện lấy điểm cao nhất

In [27]:
results = qdrant.similarity_search(
    "Năm nào học viện lấy điểm cao nhất", k=5
)
for res in results:
    # print(f"* {res.page_content} [{res.metadata}]")
    print(res)

page_content='- Thí sinh đạt giải học sinh giỏi cấp Quốc tế.
- Thí sinh đạt giải học sinh giỏi cấp Quốc gia.
- Thí sinh đạt giải Khoa học, kỹ thuật cấp Quốc tế.
- Thí sinh đạt giải Khoa học, kỹ thuật cấp Quốc gia.
- Theo từng đối tượng, xét theo thành tích giải từ cao xuống thấp.' metadata={'Header 1': 'Thông tin kỳ tuyển sinh của hoc viện kỹ thuật quân sự năm 2024', 'Header 2': '8. Các phương thức xét tuyển', 'Header 3': '8.1. Phương thức 1: Xét tuyển thẳng', '_id': '3fb86356-7a7d-4342-9737-b3c695d3b746', '_collection_name': 'thongtintuyensinh_db'}
page_content='a. Tổ hợp môn xét tuyển
Học viện KTQS xét tuyển theo 02 tổ hợp A00 (Toán, Vật lý, Hóa học) và A01 (Toán, Vật lý, Tiếng Anh).
b. Điểm xét trúng tuyển
- Điểm xét tuyển được xác định theo quy định của Quy chế tuyển sinh đại học, tuyển sinh cao đẳng ngành giáo dục mầm non của Bộ GD&ĐT (Ban hành kèm theo Thông tư 08/2022/TT-BGDĐT).' metadata={'Header 3': '8.5. Phương thức 5: Xét tuyển theo kết quả của Kỳ thi tốt nghiệp THPT năm 202